In [12]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import re
import csv
import time

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = 'https://companiesmarketcap.com'

In [6]:
def get_earnings(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    earnings = dom.xpath('//h2/strong/span')
    if earnings:
        return earnings[0].text
    else:
        return None
    
    
def get_revenue(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    revenue = dom.xpath('//h2/strong/span')
    if revenue:
        return revenue[0].text
    else:
        return None

def get_shares(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    shares = dom.xpath('//h2/strong/span')
    if shares:
        shares_txt = shares[0].text
        shares_txt = shares_txt.replace(',', '.')
        return shares_txt
    else:
        return None
    
def save_category(cat, id_azienda, writer):
    cat_name = re.sub(r'[^a-zA-Z0-9]', '', cat)
    row = [id_azienda, cat_name, hash(cat_name)]    
    writer.writerow(row)

In [8]:
def get_info_marketcap(url2, employees, id_, writer_a, writer_c):
    pre_req = time.time()
    req = requests.get(url2, headers)
    post_req = time.time()
    tot_req = post_req - pre_req
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    global shares2
    
    #Nome
    pre_name = time.time()
    name = dom.xpath('//div[@class = "company-name"]')
    post_name = time.time()
    tot_name = post_name -pre_name
    if name:
        name_txt = name[0].text
        name_txt = name_txt.replace('"', '')
        name_txt = name_txt.replace('\n', '')
    else:
        name_txt = None
    
    #Marketcap
    pre_mc = time.time()
    mc = dom.xpath('//div[text() = "Marketcap"]/../div[1]')
    post_mc = time.time()
    tot_mc = post_mc - pre_mc
    if mc:
        mc_txt = mc[0].text
    else:
        mc_txt = none
    
    #Share price
    pre_sp = time.time()
    sp = dom.xpath('//div[text() = "Share price"]/../div[1]')
    post_sp = time.time()
    tot_sp = post_sp - pre_sp
    if sp:
        sp_txt = sp[0].text
    else:
        sp = None
    
    #Company code
    pre_cc = time.time()
    cc = dom.xpath('//div[@class = "company-code"]')
    post_cc = time.time()
    tot_cc = post_cc - pre_cc
    if cc:
        cc_txt = cc[0].text
    else:
        cc_txt = None
        
    #Revenue e Earnings
    pre_links = time.time()
    links_1 = dom.xpath('//a[@class = "nav-link"]/@href')
    post_links = time.time()
    tot_links = post_links - pre_links
    for link in links_1:
        if '/earnings' in link:
            earnings = get_earnings(url + link)
        elif 'revenue' in link:
            revenue = get_revenue(url + link)
    
    #Number of shares
    pre_shares = time.time()
    links_2 = dom.xpath('//div[@class = "dropdown-menu"]/a/@href')
    post_shares = time.time()
    tot_shares = post_shares - pre_shares
    for link in links_2:
        if '/shares-outstanding' in link:
            shares2 = get_shares(url + link)
            
    #Categories
    pre_cat = time.time()
    categories = dom.xpath('//*[@id="cmkt"]/div[3]/div[1]/div[2]/div[3]/div[1]/a/text()')
    post_cat = time.time()
    tot_cat = post_cat - pre_cat
    for cat in categories:
        save_category(cat, id_, writer_c)
    
    
    employees = employees.replace(',', '.')

    row = [url2, id_, name_txt, cc_txt, mc_txt, sp_txt, earnings, revenue, shares2, employees, tot_req, tot_name,
          tot_cc, tot_mc, tot_sp, tot_links, tot_shares, tot_cat]
    writer_a.writerow(row)

In [11]:
ds_a = open('cmc-aziende-dataset.csv', 'w')
head_a = ['URL', 'ID', 'Name', 'Company code', 'Marketcap', 'Share price', 
        'Earnings', 'Revenue', 'Shares','Employees', 'Tempo request', 'Tempo estrazione Name', 'Tempo estrazione CC',
        'Tempo estrazione marketcap', 'Tempo estrazione Share price', 'Tempo estrazione Revenue/Earnings', 
        'Tempo estrazione Shares', 'Tempo estrazione Categories']
writer_a = csv.writer(ds_a)
writer_a.writerow(head_a)

ds_c = open('cmc-categorie-dataset.csv', 'w')
head_c = ['Company ID', 'Category', 'Hash']
writer_c = csv.writer(ds_c)
writer_c.writerow(head_c)

d = {}

id_ = 0
start = time.time()
for x in range(1, 15):     #(1, 15)
    start_req = time.time()
    req = requests.get(url + '/usa/largest-american-companies-by-number-of-employees/?page=' + str(x), headers)
    end_req = time.time()
    tot_req = end_req - start_req
    start_est = time.time()
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    links = dom.xpath('//a[contains(@href, "/marketcap")]/@href')
    employees = dom.xpath('//td[3]')
    end_est = time.time()
    tot_est = end_est - start_est
    d[x] = [tot_req, tot_est]
    print(x)
    for n in range(0, 100):   # (0, 100)
        id_ += 1
        url2 = url + links[n]
        get_info_marketcap(url2, employees[n].text, id_, writer_a, writer_c)
end = time.time();
ds_a.close()
ds_c.close()

print(end - start)
print(d)
print(id_, 'aziende')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
1079.712150335312
{1: [0.40468597412109375, 0.5542092323303223], 2: [0.18917369842529297, 0.1410374641418457], 3: [0.2138669490814209, 0.22008109092712402], 4: [0.20974493026733398, 0.23383021354675293], 5: [0.19745993614196777, 0.2245321273803711], 6: [0.21550464630126953, 0.16414332389831543], 7: [0.2039964199066162, 0.16255545616149902], 8: [0.2192673683166504, 0.15709710121154785], 9: [0.22065234184265137, 0.16556143760681152], 10: [0.23503589630126953, 0.1490774154663086], 11: [0.3730180263519287, 0.23321938514709473], 12: [0.2534956932067871, 0.1380774974822998], 13: [0.23784899711608887, 0.1911003589630127], 14: [0.24958205223083496, 0.2367093563079834]}
1400 aziende
